In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# กำหนดพาธของข้อมูล
size_train_dir = ""
size_val_dir = ""
size_test_dir = ""

In [ ]:
# กำหนดค่าพารามิเตอร์
batch_size = 32
img_height = 144
img_width = 256

In [ ]:
# โหลดข้อมูลจากแต่ละโฟลเดอร์
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    size_train_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    size_val_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    size_test_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

In [ ]:
# ตรวจสอบคลาสของข้อมูล
class_names = train_ds.class_names
num_classes = len(class_names)
print("Class Names:", class_names)
print("Number of Classes:", num_classes)


In [ ]:
# แสดงตัวอย่างข้อมูล
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1): 
    for i in range(9):  
        ax = plt.subplot(3, 3, i + 1)  
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])  
        plt.axis("off")
plt.show()


In [ ]:
# **Normalize Data**
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
# เพิ่ม Prefetch เพื่อประสิทธิภาพ
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import Adam

# สร้างโมเดล CNN
model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(16, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(32, activation='relu', kernel_regularizer='l2'),  # เพิ่ม L2 Regularization
    layers.Dropout(0.25),  # เพิ่ม Dropout
    layers.Dense(num_classes, activation='softmax')
])

# Compile โมเดล
model.compile(optimizer=Adam(learning_rate=0.001),  # ลด Learning Rate
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# แสดงโครงสร้างของโมเดล
model.summary()

In [ ]:
# **Train Model**
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

In [ ]:
# **Evaluate Model**
train_loss, train_accuracy = model.evaluate(train_ds)
print(f"Training Accuracy: {train_accuracy:.2f}")

val_loss, val_accuracy = model.evaluate(val_ds)
print(f"Validation Accuracy: {val_accuracy:.2f}")

test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Accuracy: {test_accuracy:.2f}")

In [ ]:
# **Plot Training Graphs**
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss during Training and Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy during Training and Validation')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# **Evaluate Model on Test Data**
y_true = []
y_pred = []

for images, labels in test_ds:
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(model.predict(images), axis=1))

In [ ]:
# **Classification Report**
print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
# **Confusion Matrix**
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.xticks(rotation=90)
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# **Predict and Visualize Results**
for images, labels in test_ds.take(1):  
    predictions = model.predict(images) 
    predicted_labels = np.argmax(predictions, axis=1)  
    true_labels = labels.numpy()  
    
    for i in range(5):
        plt.figure(figsize=(6, 3))
        plt.subplot(1, 5, i + 1)
        
        img = images[i].numpy() * 255  

        plt.imshow(img.astype("uint8"))
        plt.title(f"Pred: {class_names[predicted_labels[i]]}\nTrue: {class_names[true_labels[i]]}")
        plt.axis('off')
    plt.show()

In [ ]:
model.save("bottle_size_model.h5")